In [123]:
# Data Cleaning Example
import pandas as pd
import warnings

# Load dataset
data = pd.read_csv('collisions.csv')

#### Data quality report for categorical features

In [124]:
# initialize data quality plan that will be populated during the data quality report analysis
data_quality_plan_df = pd.DataFrame(
    index=data.columns, columns=["observations", "actions"], dtype="string"
)
data_quality_plan_df[:] = ""


In [125]:
cat_feat_names = data.select_dtypes(exclude="number").columns
cat_feat_names

Index(['Accident Date and Time', 'Road Location', 'Intersecting Road Location',
       'Road Configuration', 'Collision Configuration', 'Non Fatal Injury',
       'Fatal Injury', 'Young Demographic', 'Pedestrian Collision',
       'Aggressive Driving', 'Distracted Driving', 'Impaired Driving',
       'Bicycle Collision', 'Intersection Collision', 'Light Condition',
       'Artificial Light Condition', 'Weather Condition', 'Road Surface',
       'Road Condition', 'Unusual Environmental Circumstances 1',
       'Unusual Environmental Circumstances 2', 'Road Alignment', 'Road Grade',
       'School Purpose Related'],
      dtype='object')

In [126]:
def build_categorical_features_report(data_df):

    """Build tabular report for categorical features"""

    def _mode(df):
        return df.apply(lambda ft: ",".join(ft.mode().to_list()))

    def _mode_freq(df):
        return df.apply(lambda ft: ft.value_counts()[ft.mode()].sum())

    def _second_mode(df):
        return df.apply(lambda ft: ",".join(ft[~ft.isin(ft.mode())].mode().to_list()))

    def _second_mode_freq(df):
        return df.apply(
            lambda ft: ft[~ft.isin(ft.mode())]
            .value_counts()[ft[~ft.isin(ft.mode())].mode()]
            .sum()
        )

    stats = {
        "Count": len,
        "Miss %": lambda df: df.isna().sum() / len(df) * 100,
        "Card.": lambda df: df.nunique(),
        "Mode": _mode,
        "Mode Freq": _mode_freq,
        "Mode %": lambda df: _mode_freq(df) / len(df) * 100,
        "2nd Mode": _second_mode,
        "2nd Mode Freq": _second_mode_freq,
        "2nd Mode %": lambda df: _second_mode_freq(df) / len(df) * 100,
    }

    cat_feat_names = data_df.select_dtypes(exclude="number").columns
    cat_data_df = data_df[cat_feat_names]

    report_df = pd.DataFrame(index=cat_feat_names, columns=stats.keys())

    for stat_name, fn in stats.items():
        # NOTE: ignore warnings for empty features
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            report_df[stat_name] = fn(cat_data_df)

    return report_df


In [127]:
# build categorical feature report
cat_report_df = build_categorical_features_report(data)

In [128]:
cat_report_df

,Count,Miss %,Card.,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %
Accident Date and Time,35442,0.079002,34835,"03-09-2018 23.30,1/22/2021 7:00:00 PM,1/22/202...",24,0.067716,"01-04-2024 21.30,01-10-2024 14.00,02-08-2024 2...",72,0.203149
Road Location,35442,0.016929,11683,ROBIE ST,512,1.444614,HIGHWAY 102,501,1.413577
Intersecting Road Location,35442,45.155465,3316,HIGHWAY 102,407,1.148355,BEDFORD HWY,335,0.945206
Road Configuration,35442,2.694543,11,Non-intersection,18082,51.018566,Intersection - two or more public roads,12575,35.480503
Collision Configuration,35442,4.748603,16,Multiple vehicle - rear end,9477,26.739462,Multiple vehicle - same direction sideswipe,3686,10.400090
Non Fatal Injury,35442,86.315671,1,Yes,4850,13.684329,,0,0.000000
Fatal Injury,35442,99.779922,1,Yes,78,0.220078,,0,0.000000
Young Demographic,35442,0.000000,2,N,25161,70.992043,Y,10281,29.007957
Pedestrian Collision,35442,0.000000,2,N,34388,97.026127,Y,1054,2.973873
Aggressive Driving,35442,0.000000,2,N,24578,69.347102,Y,10864,30.652898


#### Data quality report for continuous features

In [129]:
contin_feat_names = data.select_dtypes("number").columns
contin_feat_names

Index(['OBJECTID', 'COLLISION_SK', 'Case File Number', 'Latitude WGS84',
       'Longitude WGS84', 'x', 'y'],
      dtype='object')

In [130]:
def build_continuous_features_report(data_df):

    """Build tabular report for continuous features"""

    stats = {
        "Count": len,
        "Miss %": lambda df: df.isna().sum() / len(df) * 100,
        "Card.": lambda df: df.nunique(),
        "Min": lambda df: df.min(),
        "1st Qrt.": lambda df: df.quantile(0.25),
        "Mean": lambda df: df.mean(),
        "Median": lambda df: df.median(),
        "3rd Qrt": lambda df: df.quantile(0.75),
        "Max": lambda df: df.max(),
        "Std. Dev.": lambda df: df.std(),
    }

    contin_feat_names = data_df.select_dtypes("number").columns
    continuous_data_df = data_df[contin_feat_names]

    report_df = pd.DataFrame(index=contin_feat_names, columns=stats.keys())

    for stat_name, fn in stats.items():
        # NOTE: ignore warnings for empty features
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            report_df[stat_name] = fn(continuous_data_df)

    return report_df


In [131]:
# These are helper functions for data quality report exploration


def plot_features(data_df, **kwargs):
    """Plot feature graphs in a single figure

    Args:
        data_df (pd.DataFrame): Feature values to plot
    """
    n_cols = kwargs.pop("n_cols")
    w = kwargs.pop("width", 5)
    h = kwargs.pop("height", 5)
    n_cols = min(data_df.columns.size, n_cols)
    n_rows = math.ceil(data_df.columns.size / n_cols)

    kwargs["figsize"] = (w * n_cols, h * n_rows)

    fig, axs = plt.subplots(
        n_rows,
        n_cols,
        sharex=kwargs["sharex"],
        sharey=kwargs["sharey"],
        figsize=kwargs["figsize"],
    )

    for ax in axs.flatten()[data_df.columns.size :]:
        plt.delaxes(ax)

    for ax, ft_name in zip(axs.flatten(), data_df.columns):
        data_df[ft_name].dropna().sort_values(ascending=False).plot(
            **kwargs, ax=ax, subplots=True
        )
        ax.set_title("\n".join(wrap(ft_name, 20)), pad=10)
        ax.set_xlabel(kwargs["xlabel"])

        if kwargs.get("rot"):
            plt.setp(ax.get_xticklabels(), ha="right", rotation_mode="anchor")

    fig.tight_layout()


def show_data_report(report_df, is_categorical=False, **kwargs):
    """Shows data quality report and plots graph depending on whether
    the report is for continuous features or categorical.

    Args:
        report_df (pd.DataFrame):
            Data quality report.
        is_categorical (bool, optional):
            Is the data quality report for categorical or continuous features. Defaults to continuous.
    """
    kwargs["kind"] = "bar" if is_categorical else "hist"
    kwargs["n_cols"] = kwargs.get("n_cols", 4)
    kwargs["sharex"] = kwargs.get("sharex", False)
    kwargs["sharey"] = kwargs.get("sharey", True)
    kwargs["xlabel"] = kwargs.get("xlabel", None)
    kwargs["legend"] = kwargs.get("legend", False)

    if is_categorical:
        _df = dataset_df[report_df.index].apply(pd.Series.value_counts)
        kwargs["rot"] = kwargs.get("rot", 0)
    else:
        _df = dataset_df[report_df.index]
        kwargs["bins"] = kwargs.get("bins", 50)
        if kwargs["sharex"] == True:
            kwargs["range"] = (report_df["Min"].min(), report_df["Max"].max())

    display(report_df)
    plot_features(_df, **kwargs)


In [132]:
# initialize data quality plan that will be populated during the data quality report analysis
data_quality_plan_df = pd.DataFrame(
    index=data.columns, columns=["observations", "actions"], dtype="string"
)
data_quality_plan_df[:] = ""


In [133]:
# build continuous features report
con_report_df = build_continuous_features_report(data)

In [134]:
con_report_df

,Count,Miss %,Card.,Min,1st Qrt.,Mean,Median,3rd Qrt,Max,Std. Dev.
OBJECTID,35442,0.0,35442,1.000000e+00,8.861250e+03,1.772150e+04,1.772150e+04,2.658175e+04,3.544200e+04,10231.368457
COLLISION_SK,35442,0.0,35442,2.000000e+00,2.147250e+04,4.460289e+04,3.692600e+04,7.869825e+04,9.192900e+04,29395.868074
Case File Number,35442,0.0,32643,5.000000e+00,3.819475e+04,9.877725e+04,8.328450e+04,1.307318e+05,2.215198e+07,257371.693804
Latitude WGS84,35442,0.0,13137,4.446415e+01,4.464876e+01,4.468574e+01,4.466802e+01,4.470364e+01,4.526342e+01,0.065665
Longitude WGS84,35442,0.0,14950,-6.407607e+01,-6.365464e+01,-6.359972e+01,-6.360469e+01,-6.357233e+01,-6.217746e+01,0.125639
x,35442,0.0,13106,-7.132915e+06,-7.086001e+06,-7.079887e+06,-7.080442e+06,-7.076839e+06,-6.921563e+06,13984.834260
y,35442,0.0,13106,5.537556e+06,5.566396e+06,5.572190e+06,5.569409e+06,5.574987e+06,5.663089e+06,10292.910995


#### Data Handling as per data quality plan

##### Handling missing values

In [135]:
# continuous features
missing_con_cols = con_report_df.query("`Miss %` > 0.0").index
con_report_df.loc[missing_con_cols].assign(type=data[missing_con_cols].dtypes)

,Count,Miss %,Card.,Min,1st Qrt.,Mean,Median,3rd Qrt,Max,Std. Dev.,type


In [136]:
# categorical features 
missing_cat_cols = cat_report_df.query("`Miss %` > 50.0").index
cat_report_df.loc[missing_cat_cols].assign(type=data[missing_cat_cols].dtypes)

,Count,Miss %,Card.,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %,type
Non Fatal Injury,35442,86.315671,1,Yes,4850,13.684329,,0,0.000000,object
Fatal Injury,35442,99.779922,1,Yes,78,0.220078,,0,0.000000,object
Unusual Environmental Circumstances 1,35442,93.631849,8,Animal,1062,2.996445,Limited visibility,542,1.529259,object
Unusual Environmental Circumstances 2,35442,99.147904,8,Glare or reflection,81,0.228542,Limited visibility,60,0.169291,object
School Purpose Related,35442,99.444162,2,School bus or school purpose related vehicle d...,183,0.516337,School bus or school purpose related vehicle i...,14,0.039501,object


##### filling values in missing data ( Data imputing)

In [137]:
#cat_fillna_vals = data[missing_cat_cols].mode().squeeze()
#dataset_df = data.fillna(cat_fillna_vals)

##### Handling categorical columns with cardinality one or two

In [ ]:
from datetime import datetime

# Function to parse mixed date formats
def parse_mixed_formats(date_str):
    if pd.isna(date_str):  # Handle missing values
        return None
    try:
        return datetime.strptime(date_str, "%m/%d/%Y %I:%M:%S %p")
    except ValueError:
        return datetime.strptime(date_str, "%m-%d-%Y %H.%M")
 
# Apply the parsing function to standardize dates
data['Accident Date and Time'] = data['Accident Date and Time'].apply(parse_mixed_formats)
 
 
# Convert to a consistent string format
data['Accident Date and Time'] = data['Accident Date and Time'].dt.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
from datetime import datetime


# Data Cleaning: Handle missing values, fix data types, remove duplicates, etc.
data['Accident Date and Time'] = pd.to_datetime(data['Accident Date and Time'], errors='coerce', dayfirst=True)

# Define a placeholder date, e.g., January 1, 1900
placeholder_date = datetime(1800, 1, 1)

# Fill missing values in 'Accident Date and Time' with the placeholder date

data['Accident Date and Time'].fillna(placeholder_date, inplace=True)


# Extract features like Year, Month, Day of Week, and Hour for better analysis
data['Year'] = data['Accident Date and Time'].dt.year
data['Month'] = data['Accident Date and Time'].dt.month
data['Day_of_Week'] = data['Accident Date and Time'].dt.day_name()
data['Hour'] = data['Accident Date and Time'].dt.hour

data = data[data['Year'] != 1800]

# Drop the OBJECTID column as it's just an ID
data = data.drop(columns=['Accident Date and Time'])

# Perform EDA (visualizations, correlations)

C:\Users\Ashish Kumar\AppData\Local\Temp\ipykernel_10040\1182378503.py:8: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data['Accident Date and Time'] = pd.to_datetime(data['Accident Date and Time'], errors='coerce', dayfirst=True)
C:\Users\Ashish Kumar\AppData\Local\Temp\ipykernel_10040\1182378503.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Accident Date and Time'].fillna(placeholder_date, inplace=Tru

##### The column ObjectId is just an Id , it can be dropped. Also , there are certain columns(Non Fatal Injury,Fatal Injury) where there are blank fields and a Yes , blankfield might imply No. Also columns like - Young Demographic,Pedestrian Collision,Aggressive Driving,Distracted Driving,Impaired Driving,Bicycle Collision,Intersection Collision --- these columns have only two values - Y or N.The values can be mapped to 0 and 1

In [140]:
# Data cleaning steps for "Non Fatal Injury" and "Fatal Injury" columns
injury_columns = ['Non Fatal Injury', 'Fatal Injury']
data[injury_columns] = data[injury_columns].fillna('No')  # Fill missing with 'No'



# Drop the OBJECTID column as it's just an ID
data = data.drop(columns=['OBJECTID'])
data = data.drop(columns=['Case File Number'])
data = data.drop(columns=['COLLISION_SK'])
data = data.drop(columns=['x'])
data = data.drop(columns=['y'])


#Drop 'Road Location', 'Intersecting Road Location' as we already have longitude and latitude
data =data.drop(columns=['Road Location', 'Intersecting Road Location'])

unusual_columns = ['Unusual Environmental Circumstances 2','Unusual Environmental Circumstances 1','Road Alignment',
                   'School Purpose Related','Road Grade','Day_of_Week','Road Configuration']
data[unusual_columns] = data[unusual_columns].fillna('NA') 


# Convert 'Y'/'N' binary columns to 1/0
binary_columns = ['Young Demographic', 'Pedestrian Collision', 'Aggressive Driving', 
                  'Distracted Driving', 'Impaired Driving', 'Bicycle Collision', 
                  'Intersection Collision']
data[binary_columns] = data[binary_columns].applymap(lambda x: 1 if x == 'Y' else 0)
data[injury_columns] = data[injury_columns].applymap(lambda x: 1 if x == 'Yes' else 0)

C:\Users\Ashish Kumar\AppData\Local\Temp\ipykernel_10040\2547054198.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[binary_columns] = data[binary_columns].applymap(lambda x: 1 if x == 'Y' else 0)
C:\Users\Ashish Kumar\AppData\Local\Temp\ipykernel_10040\2547054198.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[injury_columns] = data[injury_columns].applymap(lambda x: 1 if x == 'Yes' else 0)


##### Perform one hot encoding on categorical data

In [141]:
# Categorical columns for one-hot encoding
categorical_columns = ['Road Configuration','Collision Configuration', 'Light Condition', 'Artificial Light Condition', 
                       'Weather Condition', 'Road Surface', 'Road Condition', 
                       'Unusual Environmental Circumstances 1', 'Unusual Environmental Circumstances 2', 
                       'Road Alignment', 'Road Grade', 'School Purpose Related']





In [142]:
# Check unique values in each categorical column
for col in categorical_columns:
    print(f'{col} has {data[col].nunique()} unique values')

Road Configuration has 12 unique values
Collision Configuration has 16 unique values
Light Condition has 4 unique values
Artificial Light Condition has 2 unique values
Weather Condition has 8 unique values
Road Surface has 10 unique values
Road Condition has 7 unique values
Unusual Environmental Circumstances 1 has 9 unique values
Unusual Environmental Circumstances 2 has 9 unique values
Road Alignment has 3 unique values
Road Grade has 5 unique values
School Purpose Related has 3 unique values


In [143]:
# Apply one-hot encoding to these columns
data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Display first few rows of the encoded data to confirm one-hot encoding
#data_encoded.sample(5)

In [144]:
# Map Day_of_Week to digits (Monday=0, ..., Sunday=6)
day_of_week_mapping = {
    "Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3,
    "Friday": 4, "Saturday": 5, "Sunday": 6, "NA": -1
}
data['Day_of_Week'] = data['Day_of_Week'].map(day_of_week_mapping)
data_encoded['Day_of_Week'] = data_encoded['Day_of_Week'].map(day_of_week_mapping)

In [145]:
# Save the cleaned data to a CSV file
data.to_csv('cleaned_collisions_unencoded.csv', index=False)

In [146]:
data_encoded.to_csv('cleaned_collisions.csv', index=False)

In [147]:
data_encoded.shape

(35414, 91)